In [ ]:
# 영상을 받아 rppg(r,g,b) raw data, timestamp 검출
# A 코드는 산소포화도만 구할때 사용하는 코드
# B 코드는 ppg r,g,b에서 처리를 통해 chrom 방식으로 노이즈 제거한 후 ppg raw 신호 추출하는 코드
# B 코드의 결과는 r,g,b 따로가 아니라 이걸 합쳐서 하나의 PPG 신호가 나옴

# B 코드는 그냥 참고용이고 현재 방법의 산소포화도 구할때는 필요 없어요!

## A 코드 : 산소포화도만 구할때 사용한 코드

In [1]:
import cv2
import numpy as np
import os
import json
from scipy import signal
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
import csv

from utils import kcf
from utils import skinsegment

In [3]:
def run(file_path, file_num):
    is_tracking = False
    track_toler = 1
    prev_bbox = [0, 0, 0, 0]
    detect_th = 0.5
    detector = cv2.dnn.readNetFromTensorflow('model/face_detector.pb', 'model/face_detector.pbtxt') #얼굴 검출
    tracker = kcf.KCFTracker() #얼굴 tracking

    r_buffer = []
    g_buffer = []
    b_buffer = []

   # cap = cv2.VideoCapture(0, cv2.CAP_DSHOW) #카메라로 촬영하면서 바로 ppg 신호 추출할때 사용
   # h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
   # w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    file_list = os.listdir(file_path)   # 지정한 디렉토리 내 모든 파일과 디렉토리의 list return
    file_list = sorted(file_list, key= lambda x: int(os.path.splitext(x)[0].split('_')[1]))

    count = 0
    
    while True:
        # _, frame = cap.read() #카메라로 촬영하면서 바로 ppg 신호 추출할때 사용
        frame = cv2.imread('%s/%s'%(file_path, file_list[count]), cv2.IMREAD_COLOR)
        h, w = frame.shape[:2]
        
        curr_bbox = []
        # 검출된 얼굴이 존재하지 않는 경우 검출 수행
        if not is_tracking:
            blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), [104., 117., 123.], False, True)
            detector.setInput(blob)
            detections = detector.forward()
            bboxes = [detections[0, 0, i, 3:7] for i in range(detections.shape[2]) if detections[0, 0, i, 2] >= detect_th]
            if len(bboxes) > 0:
                # 가장 큰 얼굴 하나만 검출
                bboxes = sorted(bboxes, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]), reverse=True)
                bboxes = [(rect * np.array([w, h, w, h])).astype('int') for rect in bboxes]
                curr_bbox = [bboxes[0][0], bboxes[0][1], bboxes[0][2]-bboxes[0][0], bboxes[0][3]-bboxes[0][1]]  # (xs,ys,xe,ye) -> (x,y,w,h)
                
                # 트래커에 현재 얼굴 위치 등록
                tracker.init(frame, curr_bbox)
                is_tracking = True
        
        # 검출된 얼굴이 존재하는 경우 위치 업데이트
        elif is_tracking:
            is_tracking, curr_bbox = tracker.update(frame)
            curr_bbox = [curr if abs(curr-prev) > track_toler else prev for curr, prev in zip(curr_bbox, prev_bbox)]
            prev_bbox = curr_bbox
        
        try:
#             cv2.rectangle(frame, (curr_bbox[1], curr_bbox[0]), (curr_bbox[3], curr_bbox[0]), (0,0,255), 1)
            
            face = frame[curr_bbox[1]:curr_bbox[1]+curr_bbox[3], curr_bbox[0]:curr_bbox[0]+curr_bbox[0]]
            mask = skinsegment.create_skin_mask(face) # 얼굴에서 피부색 영역만 추출
            n_pixel = max(1, np.sum(mask))
            
            ''' rgb '''
            b, g, r = cv2.split(face)
            r[mask == 0] = 0
            g[mask == 0] = 0
            b[mask == 0] = 0

            r = r.astype(np.float32)
            g = g.astype(np.float32)
            b = b.astype(np.float32)

            r_mean = np.sum(r) / n_pixel
            g_mean = np.sum(g) / n_pixel
            b_mean = np.sum(b) / n_pixel

            r_buffer.append(r_mean)
            g_buffer.append(g_mean)
            b_buffer.append(b_mean)

            cv2.rectangle(frame, (curr_bbox[0], curr_bbox[1]), (curr_bbox[0]+curr_bbox[2], curr_bbox[1]+curr_bbox[3]), (0,0,255), 2)
        
        except:
            r_buffer.append(0.0)
            g_buffer.append(0.0)
            b_buffer.append(0.0) 

        cv2.imshow('frame', frame)
        count += 1
        # 파일을 다 불러왔으면 멈춤
        if len(file_list) == count :
            break           
        key = cv2.waitKey(1)
        if key == 27:
            break

    cv2.destroyAllWindows()
    
    file_time =[]
    for file in file_list:
        file_time.append(file.split('_')[-1].split('.')[0])
        
    return [r_buffer, g_buffer, b_buffer, file_time]

In [1]:
for i in range(10, 11): #(*) file_num에 맞게 불러오면 됨 #file_num은 video.ipynb 파일에서 이미지 저장할때 폴더명(=path_num)
    # RGB 신호 생성
    file_num = i
    file_path = 'images_2/%d'%file_num

    rgb_signal = run(file_path, file_num)

    #ppg rgb 신호와 그 time 엑셀에 저장
    with open('data/rppg_2/rppg%d.csv'%file_num, 'w', newline='') as f:
        wr = csv.writer(f)
        wr.writerow(rgb_signal)
        f.close()

NameError: name 'run' is not defined

## B. 처리된 PPG raw signal 구하는 코드(CHORM 활용)

In [ ]:
fs = 30
minBPM = 42 #최소 심박수 (*)
maxBPM = 240 #최대 심박수 (*)

# 신호의 추세를 없애는 함수
def detrending(data, wsize):
    try:
        if data.ndim == 1:
            data = np.expand_dims(data, axis=0)
        n_channel = data.shape[0]
        norm = signal.convolve2d(np.ones_like(data), np.ones((n_channel, wsize)), mode='same')
        mean = signal.convolve2d(data, np.ones((n_channel, wsize)), mode='same') / norm
        return (data - mean) / (mean + 1e-15)
    except ValueError:
        return data

#bandpass-filtering
def temporal_bandpass_filter(data, fs, minBPM, maxBPM):
    try:
        nyq = 60 * fs / 2
        coef_vector = signal.butter(5, [minBPM / nyq, maxBPM / nyq], 'bandpass')
        return signal.filtfilt(*coef_vector, data)
    except ValueError:
        return data

#평균 심박수 구하는 함수
def estimate_average_pulserate(data, fs, minBPM, maxBPM):
    f, pxx = signal.periodogram(data, fs=fs, window='hann')
    max_peak_idx = np.argmax(pxx)
    bpm = int(f[max_peak_idx] * 60)
    return min(max(bpm, minBPM), maxBPM)

#snr 구하는 함수
def calculate_snr(data, fs, fundamental_freq, use_harmonic=False):
    f, pxx = signal.periodogram(data, fs=fs, window='hann')

    fundamental_range = (fundamental_freq - 2, fundamental_freq + 2)
    energy_of_interest = np.sum(pxx[fundamental_range[0]:fundamental_range[1]])

    if use_harmonic:
        harmonic_freq = 2 * fundamental_freq
        harmonic_range = (harmonic_freq - 5, harmonic_freq + 5)
        energy_of_interest += np.sum(pxx[harmonic_range[0]:harmonic_range[1]])

    energy_of_remaining = np.sum(pxx) - energy_of_interest
    ratio = energy_of_interest / (energy_of_remaining + 1e-17)
    snr = 10 * np.log10(ratio)

    return snr

In [ ]:
file_num = 10 #(*) file_num에 맞게 불러오면 됨 #file_num은 video.ipynb 파일에서 이미지 저장할때 폴더명(=path_num)
file_path = 'images_2/%d'%file_num

rgb_signal = run(file_path, file_num)

# CHROM 검출
raw_signal = np.array(rgb_signal).transpose()
detrended = detrending(raw_signal, fs)
detrended = detrended.transpose()
X = 3 * detrended[0] - 2 * detrended[1]
Y = 1.5 * detrended[0] + detrended[1] - 1.5 * detrended[2]
Xf = temporal_bandpass_filter(X, fs, minBPM, maxBPM)
Yf = temporal_bandpass_filter(Y, fs, minBPM, maxBPM)
alpha = np.std(Xf) / np.std(Yf)
chrom_signal = Xf - alpha * Yf

# 주파수 스펙트럼
f, chrom_spectrum = signal.periodogram(chrom_signal, fs=fs, window='hann')

# 맥박수 추정
pr = estimate_average_pulserate(chrom_signal, fs, minBPM, maxBPM)

# 그래프 시각화
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(chrom_signal, 'b', linewidth=2)
plt.xlim([0, len(chrom_signal)])

ax = plt.subplot(1, 2, 2)
plt.plot(f, chrom_spectrum, 'k', linewidth=2)
plt.scatter(f[np.argmax(chrom_spectrum)], np.max(chrom_spectrum), c='r', marker='x')
plt.xlim([0, fs//2])
plt.text(f[np.argmax(chrom_spectrum)] + 1, np.max(chrom_spectrum), '%d bpm' % pr, fontsize=15)

plt.tight_layout()
plt.show()